# 01. 웹 크롤링 기초 (Static Crawling)

> **목표**: 인터넷 상의 데이터를 파이썬으로 수집하는 '크롤링'의 기본 원리를 이해합니다. 정적(Static) 크롤링의 대표 주자인 `requests`와 `BeautifulSoup`을 사용해봅니다.

## 1. 크롤링이란? (웹페이지 긁어오기)
우리가 웹 브라우저(Chrome)로 뉴스를 보는 것처럼, 파이썬에게 "이 사이트 좀 보여줘!"라고 시키는 것입니다.

- **정적 크롤링 (Static)**: 신문 기사처럼 내용이 변하지 않는 페이지를 가져올 때 사용합니다. (빠르고 쉬움)
- **동적 크롤링 (Dynamic)**: 인스타그램처럼 스크롤을 내릴 때마다 새로운 내용이 로딩되는 페이지에 사용합니다. (조금 느리고 복잡함)

## 2. 필수 라이브러리 (비유로 이해하기)

### (1) requests: 배달부 🛵
우리가 주소(URL)를 주면, 해당 페이지의 정보(HTML)를 받아오는 역할을 합니다.

### (2) BeautifulSoup: 번역가 🧐
`requests`가 가져온 HTML은 복잡한 외계어(태그 덩어리) 같습니다. `BeautifulSoup`은 이걸 우리가 이해하고 다루기 쉬운 형태(객체)로 바꿔줍니다.

## 3. 실습: 축구 선수 몸값 정보 가져오기 (Transfermarkt)
세계적인 축구 이적 시장 사이트인 Transfermarkt에서 선수들의 정보를 가져와봅시다.

In [ ]:
# 라이브러리 설치 (필요시 실행)
# !pip install requests beautifulsoup4 pandas

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# 1. 배달부(requests) 준비
# User-Agent: 나 사람이랑 비슷한 브라우저야! 라고 속이기 (통행증)
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}

url = "https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop"

# 요청 보내기
response = requests.get(url, headers=headers)

# 응답 코드 확인 (200이면 성공, 404면 페이지 없음, 403이면 차단됨)
print("응답 상태:", response.status_code)

> [!IMPORTANT]
> **Status Code 200**이 나와야 정상입니다. 만약 403이 나온다면 `User-Agent`를 확인해야 합니다.

In [ ]:
# 2. 번역가(BeautifulSoup) 소환
soup = BeautifulSoup(response.content, "html.parser")

# 선수 목록이 담긴 표(Table) 찾기
# HTML 구조를 보고 'tr' 태그 중 class가 'odd'나 'even'인 것을 찾습니다.
players = soup.find_all("tr", class_=["odd", "even"])

print(f"찾은 선수 수: {len(players)}명")
print("첫 번째 선수 데이터 예시:", players[0].text[:50]) # 너무 기니까 앞부분만 출력

## 4. 데이터 추출하기 (보물 찾기)
이제 `players` 리스트에서 우리가 원하는 정보(순위, 이름, 포지션, 나이, 국적, 소속팀, 시장가치)만 쏙쏙 뽑아냅시다.

In [ ]:
data_list = []

for player in players:
    info = player.find_all("td")
    
    # 각 태그의 위치(인덱스)를 확인하여 데이터 추출
    number = info[0].text.strip()       # 순위
    name = info[3].text.strip()         # 이름
    position = info[4].text.strip()     # 포지션
    age = info[5].text.strip()          # 나이
    nation = info[6].img['alt']         # 국적 (이미지의 alt 속성)
    team = info[7].img['alt']           # 소속팀
    value = info[8].text.strip()        # 시장가치
    
    data_list.append([number, name, position, age, nation, team, value])

# Pandas 데이터프레임으로 변환 (엑셀처럼 보기 좋게)
df = pd.DataFrame(data_list, columns=["Rank", "Name", "Position", "Age", "Nation", "Team", "Value"])
df.head()

## 5. 핵심 요약 (Quiz)

**Q1. 웹페이지의 주조(URL)만 주면 내용을 가져오는 '배달부' 라이브러리는?**
1. requests
2. responses
3. pandas

**Q2. 가져온 HTML 태그 덩어리를 예쁘게 해석해주는 '번역가' 라이브러리는?**
1. UglySoup
2. BeautifulSoup
3. MisoSoup

In [ ]:
# 정답 확인
print("Q1 정답: 1번 requests")
print("Q2 정답: 2번 BeautifulSoup")

## 6. ⚔️ 도전! 실습 미션
**미션: 방금 만든 `df` (데이터프레임)에서 상위 3명의 선수 이름만 출력해보세요.**

힌트: `df.head(3)` 또는 `df['Name'][:3]`

In [ ]:
# 여기에 코드를 작성하세요
